In [1]:
# Install Horovod pytorch

In [ ]:
!HOROVOD_WITH_PYTORCH=1 pip  install horovod[spark]

In [ ]:
import torch
import horovod.torch as hvd


In [ ]:
# Initialize Horovod
hvd.init()

In [ ]:

# Pin GPU to be used to process local rank (one GPU per process)
torch.cuda.set_device(hvd.local_rank())



In [ ]:
# Define dataset...
train_dataset = ...
# Partition dataset among workers using DistributedSampler
train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset, num_replicas=hvd.size(), rank=hvd.rank())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=..., sampler=train_sampler)

In [2]:
!pwd


/home/ec2-user/SageMaker/distributed_computing


In [ ]:


# Build model...
model = ...
model.cuda()


In [ ]:

optimizer = optim.SGD(model.parameters())

In [ ]:
# Add Horovod Distributed Optimizer
optimizer = hvd.DistributedOptimizer(optimizer, named_parameters=model.named_parameters())


In [ ]:

# Broadcast parameters from rank 0 to all other processes.
hvd.broadcast_parameters(model.state_dict(), root_rank=0)

In [ ]:
for epoch in range(100):
   for batch_idx, (data, target) in enumerate(train_loader):
       optimizer.zero_grad()
       output = model(data)
       loss = F.nll_loss(output, target)
       loss.backward()
       optimizer.step()
       if batch_idx % args.log_interval == 0:
           print('Train Epoch: {} [{}/{}]\tLoss: {}'.format(
               epoch, batch_idx * len(data), len(train_sampler), loss.item()))